# Milestone 2

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import sklearn

## Load data

In [4]:
# Run this if you use colab and have the dataset uploaded on Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Use this path if you aren't using colab:
# base_path = './detection+of+iot+botnet+attacks+n+baiot/'
base_path = 'drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/'

folders = ['SimpleHome_XCS7_1003_WHT_Security_Camera', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'Provision_PT_838_Security_Camera', 'Provision_PT_737E_Security_Camera', 'Philips_B120N10_Baby_Monitor', 'Ennio_Doorbell', 'Ecobee_Thermostat', 'Danmini_Doorbell']

# 1. Load benign traffic
benign_traffic = pd.DataFrame()
for folder in folders:
  curr_csv = pd.read_csv(base_path + folder + '/benign_traffic.csv')
  curr_csv['Device'] = folder
  benign_traffic = pd.concat([benign_traffic, curr_csv])
# benign_traffic1 = pd.read_csv(base_path + 'SimpleHome_XCS7_1003_WHT_Security_Camera/benign_traffic.csv')
# benign_traffic2 = pd.read_csv(base_path + 'Samsung_SNH_1011_N_Webcam/benign_traffic.csv')
# benign_traffic = pd.concat([benign_traffic1, benign_traffic2])
benign_traffic

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,Device
0,1.000000,60.0,0.000000e+00,1.000000,60.0,0.000000e+00,1.000000,60.000000,0.000000,1.000000,...,0.000000e+00,0.0,1.000000,60.000000,0.000000e+00,60.000000,0.000000e+00,0.000000e+00,0.000000e+00,SimpleHome_XCS7_1003_WHT_Security_Camera
1,1.000000,60.0,0.000000e+00,1.000000,60.0,0.000000e+00,1.000000,60.000000,0.000000,1.000000,...,0.000000e+00,0.0,1.061357,60.000000,9.540000e-07,60.000000,9.090000e-13,0.000000e+00,0.000000e+00,SimpleHome_XCS7_1003_WHT_Security_Camera
2,1.000000,60.0,0.000000e+00,1.000000,60.0,0.000000e+00,1.000000,60.000000,0.000000,1.000000,...,0.000000e+00,0.0,1.000000,60.000000,0.000000e+00,60.000000,0.000000e+00,0.000000e+00,0.000000e+00,SimpleHome_XCS7_1003_WHT_Security_Camera
3,1.000000,590.0,0.000000e+00,1.000000,590.0,0.000000e+00,1.000000,590.000000,0.000000,1.000000,...,0.000000e+00,0.0,5.832783,388.850426,9.199164e+01,388.850426,8.462461e+03,0.000000e+00,0.000000e+00,SimpleHome_XCS7_1003_WHT_Security_Camera
4,1.927179,590.0,0.000000e+00,1.955648,590.0,0.000000e+00,1.984992,590.000000,0.000000,1.998489,...,0.000000e+00,0.0,6.831901,418.293119,1.108120e+02,418.293119,1.227931e+04,0.000000e+00,0.000000e+00,SimpleHome_XCS7_1003_WHT_Security_Camera
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49543,1.000000,102.0,0.000000e+00,1.000000,102.0,1.820000e-12,1.000009,101.999633,0.015405,2.270210,...,-1.570000e-30,0.0,4.218824,102.000000,0.000000e+00,144.249783,1.820000e-12,5.970000e-23,0.000000e+00,Danmini_Doorbell
49544,1.999976,102.0,0.000000e+00,1.999986,102.0,1.820000e-12,2.000004,101.999816,0.007702,3.270209,...,-1.580000e-44,0.0,4.218838,102.000000,0.000000e+00,144.249783,3.640000e-12,-1.100000e-29,0.000000e+00,Danmini_Doorbell
49545,2.999872,102.0,3.640000e-12,2.999923,102.0,1.820000e-12,2.999983,101.999878,0.005135,4.270206,...,-8.330000e-45,0.0,4.179949,102.000000,1.910000e-06,144.249783,5.140000e-12,8.230000e-29,2.260000e-17,Danmini_Doorbell
49546,3.999664,102.0,3.640000e-12,3.999798,102.0,0.000000e+00,3.999942,101.999908,0.003851,5.270200,...,4.980000e-69,0.0,4.219537,102.000000,1.350000e-06,144.249783,1.820000e-12,5.960000e-29,0.000000e+00,Danmini_Doorbell


In [18]:
!pip install patool
import patoolib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 kB 1.9 MB/s eta 0:00:00


In [21]:
for folder in folders:
  patoolib.extract_archive(base_path+folder+"/gafgyt_attacks.rar", outdir=base_path+folder)


INFO patool: Extracting drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/SimpleHome_XCS7_1003_WHT_Security_Camera/gafgyt_attacks.rar ...
INFO:patool:Extracting drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/SimpleHome_XCS7_1003_WHT_Security_Camera/gafgyt_attacks.rar ...
INFO patool: running /usr/bin/unrar x -kb -or -- /content/drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/SimpleHome_XCS7_1003_WHT_Security_Camera/gafgyt_attacks.rar
INFO:patool:running /usr/bin/unrar x -kb -or -- /content/drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/SimpleHome_XCS7_1003_WHT_Security_Camera/gafgyt_attacks.rar
INFO patool: ... drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/SimpleHome_XCS7_1003_WHT_Security_Camera/gafgyt_attacks.rar extracted to `drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/SimpleHome_XCS7_1003_WHT_Security_Camera'.
INFO:patool:... drive/MyDrive/detection+of+iot+botnet+attacks+n+baiot/SimpleHome_XCS7_1003_WHT_Security_Camera/gafgyt_attacks.rar ex

In [27]:
test = pd.read_csv(base_path+folders[2]+'/junk.csv')
test

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
0,1.000000,98.000000,0.000000,1.000000,98.000000,0.000000e+00,1.000000,98.000000,0.000000e+00,1.000000,...,0.000000e+00,0.0,0.0,1.000000,98.0,0.000000e+00,98.000000,0.000000e+00,0.0,0.0
1,1.029191,98.000000,0.000000,1.119992,98.000000,1.818989e-12,1.493231,98.000000,3.637979e-12,1.931762,...,0.000000e+00,0.0,0.0,1.992957,98.0,0.000000e+00,138.592929,0.000000e+00,0.0,0.0
2,1.077270,68.295282,68.180692,1.236877,72.128396,1.585514e+02,1.889672,81.065843,2.551274e+02,2.834273,...,0.000000e+00,0.0,0.0,1.000000,66.0,0.000000e+00,114.039467,0.000000e+00,0.0,0.0
3,2.038100,71.094319,42.860737,2.209694,72.975393,8.766659e+01,2.875726,78.608779,1.777339e+02,3.832174,...,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000e+00,74.000000,0.000000e+00,0.0,0.0
4,3.000117,72.062842,30.450564,3.184892,73.297101,6.036696e+01,3.864926,77.416316,1.358213e+02,4.830732,...,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000e+00,74.000000,0.000000e+00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28300,163.002961,74.132261,3.156770,273.300355,74.091494,2.187882e+00,759.282825,74.060869,5.117341e+00,6723.413700,...,0.000000e+00,0.0,0.0,1.682391,74.0,9.536743e-07,95.268043,9.094947e-13,0.0,0.0
28301,163.062862,74.131450,3.137517,273.353526,74.091160,2.179909e+00,759.404984,74.060789,5.110607e+00,6723.635971,...,0.000000e+00,0.0,0.0,1.682383,74.0,9.536743e-07,95.268043,9.094947e-13,0.0,0.0
28302,164.058954,74.130649,3.118497,274.349596,74.090827,2.171993e+00,760.401344,74.060709,5.103891e+00,6724.632748,...,1.818989e-12,0.0,0.0,1.682383,74.0,1.348699e-06,95.268043,1.818989e-12,0.0,0.0
28303,163.596838,74.129850,3.099539,273.879945,74.090496,2.164093e+00,760.041126,74.060629,5.097181e+00,6724.428866,...,0.000000e+00,0.0,0.0,1.682371,74.0,0.000000e+00,95.268043,0.000000e+00,0.0,0.0


# New Section